## **[Optional] I2I <sup>Image-to-image</sup> Pipelines**

> Original Source: https://huggingface.co/docs/diffusers/v0.33.1/en/using-diffusers/sdxl_turbo

```
> Stable Diffusion XL
> Stable Diffusion XL Turbo
> Kandinsky
> IP-Adapter
> Perturbed-Attention Guidance(PAG)
> ControlNet
> Latent Consistency Model(LCM)
```

-----
### **Stable Diffusion XL**
- `Stable Diffusion XL (SDXL)` is a powerful text-to-image generation model that iterates on the previous Stable Diffusion models in three key ways:
  - The `UNet` is 3x larger and `SDXL` combines a second text encoder (`OpenCLIP ViT-bigG/14`) with the original text encoder to significantly increase the number of parameters
  - Introduces size and crop-conditioning to preserve training data from being discarded and gain more control over how a generated image should be cropped
  - Introduces a two-stage model process; the base model (can also be run as a standalone model) generates an image as an input to the refiner model which adds additional high-quality details

- Install:
```
pip install -q diffusers transformers accelerate invisible-watermark>=0.2.0
```

- We recommend installing the `invisible-watermark` library to help identify images that are generated. If the invisible-watermark library is installed, it is used by default. To disable the watermarker:

```
pipeline = StableDiffusionXLPipeline.from_pretrained(..., add_watermarker=False)
```

#### Load model checkpoints
- Model weights may be stored in separate subfolders on the Hub or locally, in which case, you should use the `from_pretrained()` method:

In [ ]:
from diffusers import StableDiffusionXLPipeline
from diffusers import AutoPipelineForText2Image

from diffusers import StableDiffusionXLImg2ImgPipeline, AutoPipelineForImage2Image
from diffusers import StableDiffusionXLInpaintPipeline, AutoPipelineForInpainting

from diffusers import DiffusionPipeline

- For image-to-image, `SDXL` works especially well with image sizes between 768x768 and 1024x1024. Pass an initial image, and a text prompt to condition the image with:

In [ ]:
pipeline_text2image = AutoPipelineForText2Image.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0", torch_dtype=torch.float16, variant="fp16", use_safetensors=True
).to("cuda")

# use from_pipe to avoid consuming additional memory when loading a checkpoint
pipeline = AutoPipelineForImage2Image.from_pipe(pipeline_text2image).to("cuda")

url = "https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/sdxl-text2img.png"
init_image = load_image(url)
prompt = "a dog catching a frisbee in the jungle"
image = pipeline(prompt, image=init_image, strength=0.8, guidance_scale=10.5).images[0]
make_image_grid([init_image, image], rows=1, cols=2)

-----
### **Stable Diffusion XL Turbo**
- `SDXL Turbo` is an adversarial time-distilled `Stable Diffusion XL (SDXL)` model capable of running inference in as little as 1 step.

<br>

- Model weights may be stored in separate subfolders on the Hub or locally, in which case, you should use the `from_pretrained()` method:

In [ ]:
from diffusers import AutoPipelineForText2Image
from diffusers import StableDiffusionXLPipeline, EulerAncestralDiscreteScheduler

from diffusers import AutoPipelineForImage2Image
from diffusers.utils import load_image, make_image_grid

- For image-to-image generation, make sure that `num_inference_steps * strength` is larger or equal to 1.
  - The image-to-image pipeline will run for `int(num_inference_steps * strength)` steps.

In [ ]:
pipeline_text2image = AutoPipelineForText2Image.from_pretrained("stabilityai/sdxl-turbo", torch_dtype=torch.float16, variant="fp16")
pipeline_text2image = pipeline_text2image.to("cuda")

# use from_pipe to avoid consuming additional memory when loading a checkpoint
pipeline_image2image = AutoPipelineForImage2Image.from_pipe(pipeline_text2image).to("cuda")

init_image = load_image("https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/cat.png")
init_image = init_image.resize((512, 512))

prompt = "cat wizard, gandalf, lord of the rings, detailed, fantasy, cute, adorable, Pixar, Disney, 8k"

image = pipeline_image2image(prompt, image=init_image, strength=0.5, guidance_scale=0.0, num_inference_steps=2).images[0]
make_image_grid([init_image, image], rows=1, cols=2)

-----
### **Kandinsky**
- The Kandinsky models are a series of multilingual text-to-image generation models.
  - The `Kandinsky 2.0` model uses two multilingual text encoders and concatenates those results for the UNet.
  - `Kandinsky 2.1` changes the architecture to include an image prior model (CLIP) to generate a mapping between text and image embeddings and uses a `Modulating Quantized Vectors (MoVQ)` decoder - which adds a spatial conditional normalization layer to increase photorealism - to decode the latents into images.
  - `Kandinsky 2.2` improves on the previous model by replacing the image encoder of the image prior model with a larger `CLIP-ViT-G` model to improve quality.
    - The only difference with `Kandinsky 2.1` is `Kandinsky 2.2` doesn’t accept prompt as an input when decoding the latents. Instead, `Kandinsky 2.2` only accepts `image_embeds` during decoding.
  - `Kandinsky 3` simplifies the architecture and shifts away from the two-stage generation process involving the prior model and diffusion model and uses `Flan-UL2` to encode text, a `UNet` with BigGan-deep blocks, and `Sber-MoVQGAN` to decode the latents into images.
    - Text understanding and generated image quality are primarily achieved by using a larger text encoder and UNet.


In [2]:
from diffusers import KandinskyPriorPipeline, KandinskyPipeline
from diffusers import KandinskyV22PriorPipeline, KandinskyV22Pipeline
from diffusers import Kandinsky3Pipeline

from diffusers import AutoPipelineForText2Image
from diffusers import AutoPipelineForImage2Image
from diffusers import AutoPipelineForInpainting

from diffusers import KandinskyImg2ImgPipeline, KandinskyPriorPipeline
from diffusers import KandinskyV22Img2ImgPipeline
from diffusers import Kandinsky3Img2ImgPipeline

from diffusers import KandinskyInpaintPipeline
from diffusers import KandinskyV22InpaintPipeline, KandinskyV22PriorPipeline, KandinskyV22ControlnetPipeline

from diffusers import KandinskyV22PriorEmb2EmbPipeline, KandinskyV22ControlnetImg2ImgPipeline

from diffusers.models.attention_processor import AttnAddedKVProcessor2_0

from diffusers.utils import load_image
from diffusers.utils import make_image_grid

- For image-to-image, pass the initial image and text prompt to condition the image to the pipeline.

In [ ]:
prior_pipeline = KandinskyPriorPipeline.from_pretrained("kandinsky-community/kandinsky-2-2-prior", torch_dtype=torch.float16, use_safetensors=True).to("cuda")
pipeline = KandinskyV22Img2ImgPipeline.from_pretrained("kandinsky-community/kandinsky-2-2-decoder", torch_dtype=torch.float16, use_safetensors=True).to("cuda")

In [ ]:
url = "https://raw.githubusercontent.com/CompVis/stable-diffusion/main/assets/stable-samples/img2img/sketch-mountains-input.jpg"
original_image = load_image(url)
original_image = original_image.resize((768, 512))

- Generate the `image_embeds` and `negative_image_embeds` with the prior pipeline
  - Pass the original image, and all the prompts and embeddings to the pipeline to generate an image:

In [ ]:
prompt = "A fantasy landscape, Cinematic lighting"
negative_prompt = "low quality, bad quality"

image_embeds, negative_image_embeds = prior_pipeline(prompt, negative_prompt).to_tuple()

image = pipeline(image=original_image, image_embeds=image_embeds, negative_image_embeds=negative_image_embeds, height=768, width=768, strength=0.3).images[0]
make_image_grid([original_image.resize((512, 512)), image.resize((512, 512))], rows=1, cols=2)

- Diffusers also provides an end-to-end API with the `KandinskyImg2ImgCombinedPipeline` and `KandinskyV22Img2ImgCombinedPipeline`, meaning you don’t have to separately load the prior and image-to-image pipeline.
  - The combined pipeline automatically loads both the prior model and the decoder.
  - You can still set different values for the prior pipeline with the `prior_guidance_scale` and `prior_num_inference_steps` parameters if you want.

- Use the `AutoPipelineForImage2Image` to automatically call the combined pipelines under the hood:

In [ ]:
pipeline = AutoPipelineForImage2Image.from_pretrained("kandinsky-community/kandinsky-2-1", torch_dtype=torch.float16, use_safetensors=True)
pipeline.enable_model_cpu_offload()

prompt = "A fantasy landscape, Cinematic lighting"
negative_prompt = "low quality, bad quality"

url = "https://raw.githubusercontent.com/CompVis/stable-diffusion/main/assets/stable-samples/img2img/sketch-mountains-input.jpg"
original_image = load_image(url)

original_image.thumbnail((768, 768))

image = pipeline(prompt=prompt, negative_prompt=negative_prompt, image=original_image, strength=0.3).images[0]
make_image_grid([original_image.resize((512, 512)), image.resize((512, 512))], rows=1, cols=2)

#### ControlNet
- ControlNet enables conditioning large pretrained diffusion models with additional inputs such as a depth map or edge detection.
  - You can condition `Kandinsky 2.2` with a depth map so the model understands and preserves the structure of the depth image.
 
- Use the depth-estimation Pipeline from `Transformers` to process the image and retrieve the depth map:

In [ ]:
img = load_image(
    "https://huggingface.co/datasets/hf-internal-testing/diffusers-images/resolve/main/kandinskyv22/cat.png"
).resize((768, 768))

def make_hint(image, depth_estimator):
    image = depth_estimator(image)["depth"]
    image = np.array(image)
    image = image[:, :, None]
    image = np.concatenate([image, image, image], axis=2)
    detected_map = torch.from_numpy(image).float() / 255.0
    hint = detected_map.permute(2, 0, 1)
    return hint

depth_estimator = pipeline("depth-estimation")
hint = make_hint(img, depth_estimator).unsqueeze(0).half().to("cuda")

- `KandinskyV22PriorEmb2EmbPipeline` to generate the image embeddings from a text prompt and an image
- `KandinskyV22ControlnetImg2ImgPipeline` to generate an image from the initial image and the image embeddings

- Process and extract a depth map of an initial image of a cat with the depth-estimation Pipeline from `Transformers`:

In [ ]:
prior_pipeline = KandinskyV22PriorEmb2EmbPipeline.from_pretrained(
    "kandinsky-community/kandinsky-2-2-prior", torch_dtype=torch.float16, use_safetensors=True
).to("cuda")

pipeline = KandinskyV22ControlnetImg2ImgPipeline.from_pretrained(
    "kandinsky-community/kandinsky-2-2-controlnet-depth", torch_dtype=torch.float16
).to("cuda")

- Pass a text prompt and the initial image to the prior pipeline to generate the image embeddings:

In [ ]:
prompt = "A robot, 4k photo"
negative_prior_prompt = "lowres, text, error, cropped, worst quality, low quality, jpeg artifacts, ugly, duplicate, morbid, mutilated, out of frame, extra fingers, mutated hands, poorly drawn hands, poorly drawn face, mutation, deformed, blurry, dehydrated, bad anatomy, bad proportions, extra limbs, cloned face, disfigured, gross proportions, malformed limbs, missing arms, missing legs, extra arms, extra legs, fused fingers, too many fingers, long neck, username, watermark, signature"

generator = torch.Generator(device="cuda").manual_seed(43)

img_emb = prior_pipeline(prompt=prompt, image=img, strength=0.85, generator=generator)
negative_emb = prior_pipeline(prompt=negative_prior_prompt, image=img, strength=1, generator=generator)

image = pipeline(image=img, strength=0.5, image_embeds=img_emb.image_embeds, negative_image_embeds=negative_emb.image_embeds, hint=hint, num_inference_steps=50, generator=generator, height=768, width=768).images[0]
make_image_grid([img.resize((512, 512)), image.resize((512, 512))], rows=1, cols=2)

-----
### **IP-Adapter**
- `IP-Adapter` is an image prompt adapter that can be plugged into diffusion models to enable image prompting without any changes to the underlying model.
  - This adapter can be reused with other models finetuned from the same base model and it can be combined with other adapters like ControlNet.
  - The key idea behind `IP-Adapter` is the decoupled cross-attention mechanism which adds a separate cross-attention layer just for image features instead of using the same cross-attention layer for both text and image features.
    - This allows the model to learn more image-specific features.
   
<br>

- `set_ip_adapter_scale()` method controls the amount of text or image conditioning to apply to the model.
  - A value of `1.0` means the model is only conditioned on the image prompt.
  - Lowering this value encourages the model to produce more diverse images, but they may not be as aligned with the image prompt.
  - Typically, a value of 0.5 achieves a good balance between the two prompt types and produces good results.

- Try adding `low_cpu_mem_usage=True` to the `load_ip_adapter()` method to speed up the loading

In [ ]:
from diffusers import AutoPipelineForText2Image
from diffusers import AutoPipelineForImage2Image
from diffusers import AutoPipelineForInpainting

from diffusers.image_processor import IPAdapterMaskProcessor
from diffusers import StableDiffusionPipeline, DDIMScheduler, AutoPipelineForText2Image
from diffusers import DiffusionPipeline, LCMScheduler

from insightface.app import FaceAnalysis
from insightface.utils import face_align

from transformers import CLIPVisionModelWithProjection

- `IP-Adapter` can also help with image-to-image by guiding the model to generate an image that resembles the original image and the image prompt.
  - Pass the original image and the IP-Adapter image prompt to the pipeline to generate an image.
  - Providing a text prompt to the pipeline is optional, but in this example, a text prompt is used to increase image quality.

In [ ]:
pipeline = AutoPipelineForImage2Image.from_pretrained("stabilityai/stable-diffusion-xl-base-1.0", torch_dtype=torch.float16).to("cuda")
pipeline.load_ip_adapter("h94/IP-Adapter", subfolder="sdxl_models", weight_name="ip-adapter_sdxl.bin")
pipeline.set_ip_adapter_scale(0.6)

In [ ]:
image = load_image("https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/ip_adapter_bear_1.png")
ip_image = load_image("https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/ip_adapter_bear_2.png")

generator = torch.Generator(device="cpu").manual_seed(4)
images = pipeline(
    prompt="best quality, high quality",
    image=image,
    ip_adapter_image=ip_image,
    generator=generator,
    strength=0.6,
).images
images[0]

----
### **Perturbed-Attention Guidance(PAG)**
- `Perturbed-Attention Guidance(PAG)` is a new diffusion sampling guidance that improves sample quality across both unconditional and conditional settings, achieving this without requiring further training or the integration of external modules.
  - `PAG` is designed to progressively enhance the structure of synthesized samples throughout the denoising process by considering the self-attention mechanisms’ ability to capture structural information.
  - It involves generating intermediate samples with degraded structure by substituting selected self-attention maps in diffusion U-Net with an identity matrix, and guiding the denoising process away from these degraded samples.

<br>

- You can apply `PAG` to the `StableDiffusionXLPipeline` for tasks such as text-to-image, image-to-image, and inpainting.
- To enable `PAG` for a specific task, load the pipeline using the AutoPipeline API with the `enable_pag=True` flag and the `pag_applied_layers` argument.

In [ ]:
from diffusers import AutoPipelineForText2Image, ControlNetModel
from diffusers import AutoPipelineForImage2Image
from diffusers import AutoPipelineForInpainting

from transformers import CLIPVisionModelWithProjection

- It is also very easy to directly switch from a text-to-image pipeline to `PAG` enabled image-to-image pipeline
  - If you have a `PAG` enabled text-to-image pipeline, you can directly switch to a image-to-image pipeline with `PAG` still enabled

In [ ]:
pipeline = AutoPipelineForImage2Image.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    enable_pag=True,
    pag_applied_layers=["mid"],
    torch_dtype=torch.float16
)
pipeline.enable_model_cpu_offload()

In [ ]:
pipeline_t2i = AutoPipelineForImage2Image.from_pretrained("stabilityai/stable-diffusion-xl-base-1.0", torch_dtype=torch.float16)
pipeline = AutoPipelineForImage2Image.from_pipe(pipeline_t2i, enable_pag=True)

pipeline_pag = AutoPipelineForText2Image.from_pretrained("stabilityai/stable-diffusion-xl-base-1.0", torch_dtype=torch.float16)
pipeline = AutoPipelineForImage2Image.from_pipe(pipeline_t2i, enable_pag=True)

pipeline_pag = AutoPipelineForText2Image.from_pretrained("stabilityai/stable-diffusion-xl-base-1.0", enable_pag=True, torch_dtype=torch.float16)
pipeline = AutoPipelineForImage2Image.from_pipe(pipeline_t2i)

In [ ]:
pag_scales =  4.0
guidance_scales = 7.0

url = "https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/sdxl-text2img.png"
init_image = load_image(url)
prompt = "a dog catching a frisbee in the jungle"

generator = torch.Generator(device="cpu").manual_seed(0)
image = pipeline(
    prompt,
    image=init_image,
    strength=0.8,
    guidance_scale=guidance_scale,
    pag_scale=pag_scale,
    generator=generator).images[0]

----
### **ControlNet**
- `ControlNet` is a type of model for controlling image diffusion models by conditioning the model with an additional input image.
  - There are many types of conditioning inputs (canny edge, user sketching, human pose, depth, and more) you can use to control a diffusion model.
  - This is hugely useful because it affords you greater control over image generation, making it easier to generate specific images without experimenting with different text prompts or denoising values as much.

- A `ControlNet` model has two sets of weights (or blocks) connected by a zero-convolution layer:
  - a locked copy keeps everything a large pretrained diffusion model has learned
  - a trainable copy is trained on the additional conditioning input

- Since the locked copy preserves the pretrained model, training and implementing a `ControlNet` on a new conditioning input is as fast as finetuning any other model because you aren’t training the model from scratch.

In [5]:
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel, UniPCMultistepScheduler
from diffusers import StableDiffusionControlNetImg2ImgPipeline
from diffusers import StableDiffusionControlNetInpaintPipeline

from diffusers import StableDiffusionXLControlNetPipeline, AutoencoderKL

- With `ControlNet`, you can pass an additional conditioning input to guide the model.
  - Let’s condition the model with a depth map, an image which contains spatial information.
  - `ControlNet` can use the depth map as a control to guide the model to generate an image that preserves spatial information.

- Use the `StableDiffusionControlNetImg2ImgPipeline` for this task, which is different from the `StableDiffusionControlNetPipeline` because it allows you to pass an initial image as the starting point for the image generation process.

- Load an image and use the depth-estimation Pipeline from Transformers to extract the depth map of an image:

In [ ]:
image = load_image(
    "https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/controlnet-img2img.jpg"
)

def get_depth_map(image, depth_estimator):
    image = depth_estimator(image)["depth"]
    image = np.array(image)
    image = image[:, :, None]
    image = np.concatenate([image, image, image], axis=2)
    detected_map = torch.from_numpy(image).float() / 255.0
    depth_map = detected_map.permute(2, 0, 1)
    return depth_map

depth_estimator = pipeline("depth-estimation")
depth_map = get_depth_map(image, depth_estimator).unsqueeze(0).half().to("cuda")

- Load a `ControlNet` model conditioned on depth maps and pass it to the `StableDiffusionControlNetImg2ImgPipeline`.
  - Use the faster `UniPCMultistepScheduler` and enable model offloading to speed up inference and reduce memory usage.

In [ ]:
controlnet = ControlNetModel.from_pretrained("lllyasviel/control_v11f1p_sd15_depth", torch_dtype=torch.float16, use_safetensors=True)
pipe = StableDiffusionControlNetImg2ImgPipeline.from_pretrained(
    "stable-diffusion-v1-5/stable-diffusion-v1-5", controlnet=controlnet, torch_dtype=torch.float16, use_safetensors=True
)

pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)
pipe.enable_model_cpu_offload()

output = pipe(
    "lego batman and robin", image=image, control_image=depth_map,
).images[0]
make_image_grid([image, output], rows=1, cols=2)

----
### **Latent Consistency Model(LCMs)**
- `Latent Consistency Models (LCMs)` enable fast high-quality image generation by directly predicting the reverse diffusion process in the latent rather than pixel space.
  - `LCMs` try to predict the noiseless image from the noisy image in contrast to typical diffusion models that iteratively remove noise from the noisy image.
  - By avoiding the iterative sampling process, `LCMs` are able to generate high-quality images in 2-4 steps instead of 20-30 steps.

- `LCMs` are distilled from pretrained models which requires ~32 hours of A100 compute.
  - To speed this up, `LCM-LoRAs` train a `LoRA` adapter which have much fewer parameters to train compared to the full model.
  - The `LCM-LoRA` can be plugged into a diffusion model once it has been trained.

In [ ]:
from diffusers import StableDiffusionXLPipeline, UNet2DConditionModel, LCMScheduler, AutoPipelineForImage2Image
from diffusers import DiffusionPipeline

- To use `LCMs` for image-to-image, you need to load the `LCM` checkpoint for your supported model into `UNet2DConditionModel` and replace the scheduler with the `LCMScheduler`.

In [ ]:
unet = UNet2DConditionModel.from_pretrained(
    "SimianLuo/LCM_Dreamshaper_v7",
    subfolder="unet",
    torch_dtype=torch.float16,
)

pipe = AutoPipelineForImage2Image.from_pretrained(
    "Lykon/dreamshaper-7",
    unet=unet,
    torch_dtype=torch.float16,
    variant="fp16",
).to("cuda")
pipe.scheduler = LCMScheduler.from_config(pipe.scheduler.config)

init_image = load_image("https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/img2img-init.png")
prompt = "Astronauts in a jungle, cold color palette, muted colors, detailed, 8k"
generator = torch.manual_seed(0)
image = pipe(
    prompt,
    image=init_image,
    num_inference_steps=4,
    guidance_scale=7.5,
    strength=0.5,
    generator=generator
).images[0]
image

- To use `LCM-LoRAs` for image-to-image, you need to replace the scheduler with the `LCMScheduler` and load the `LCM-LoRA` weights with the `load_lora_weights()` method.

In [ ]:
pipe = AutoPipelineForImage2Image.from_pretrained(
    "Lykon/dreamshaper-7",
    torch_dtype=torch.float16,
    variant="fp16",
).to("cuda")

pipe.scheduler = LCMScheduler.from_config(pipe.scheduler.config)

pipe.load_lora_weights("latent-consistency/lcm-lora-sdv1-5")

init_image = load_image("https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/img2img-init.png")
prompt = "Astronauts in a jungle, cold color palette, muted colors, detailed, 8k"

generator = torch.manual_seed(0)
image = pipe(
    prompt,
    image=init_image,
    num_inference_steps=4,
    guidance_scale=1,
    strength=0.6,
    generator=generator
).images[0]
image